<a href="https://colab.research.google.com/github/jockyuiz/3D-Machine-Learning/blob/master/GRU_Stock.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [3]:
#############################
# 1. Improved GRU Module
#############################

class ImprovedGRUCell(nn.Module):
    """
    改进版 GRU 单步 Cell（Gated Recurrent Unit）
    其中：
      - Update Gate (更新门): 控制新旧信息的融合
      - 使用 Attention (注意力机制) 替换传统的 Reset Gate (重置门)
    """
    def __init__(self, input_size, hidden_size, attn_size):
        super(ImprovedGRUCell, self).__init__()
        self.hidden_size = hidden_size

        # 更新门参数
        self.W_z = nn.Linear(input_size, hidden_size)
        self.U_z = nn.Linear(hidden_size, hidden_size)

        # 候选隐藏状态参数
        self.W_h = nn.Linear(input_size, hidden_size)
        self.U_h = nn.Linear(hidden_size, hidden_size)
        self.b_h = nn.Parameter(torch.zeros(hidden_size))

        # 注意力机制参数（用于替代 reset gate）
        # 将上一时刻隐藏状态作为 Query，当前输入作为 Key 和 Value
        self.W_q = nn.Linear(hidden_size, attn_size)
        self.W_k = nn.Linear(input_size, attn_size)
        self.W_v = nn.Linear(input_size, hidden_size)
        self.attn_scale = torch.sqrt(torch.tensor(attn_size, dtype=torch.float32))

    def forward(self, x, h_prev):
        """
        x: 当前输入，形状 (batch, input_size)
        h_prev: 上一时刻隐藏状态，形状 (batch, hidden_size)
        """
        # 更新门计算：z = sigmoid(W_z * x + U_z * h_prev)
        z = torch.sigmoid(self.W_z(x) + self.U_z(h_prev))

        # 注意力机制替代重置门
        q = self.W_q(h_prev)      # Query, 形状 (batch, attn_size)
        k = self.W_k(x)           # Key,     形状 (batch, attn_size)
        v = self.W_v(x)           # Value,   形状 (batch, hidden_size)
        # 计算缩放点积注意力分数 (Scaled Dot-Product Attention)
        attn_score = torch.sum(q * k, dim=-1, keepdim=True) / self.attn_scale  # (batch, 1)
        # 使用 sigmoid 得到注意力权重（简化版，不跨时间步聚合）
        alpha = torch.sigmoid(attn_score)  # (batch, 1)
        r_prime = alpha * v  # 得到新的“重置门”输出，形状 (batch, hidden_size)

        # 计算候选隐藏状态：h_tilde = tanh(W_h * x + r_prime ⊙ (U_h * h_prev) + b_h)
        h_tilde = torch.tanh(self.W_h(x) + r_prime * self.U_h(h_prev) + self.b_h)

        # 最终隐藏状态更新：h = (1 - z) ⊙ h_prev + z ⊙ h_tilde
        h = (1 - z) * h_prev + z * h_tilde
        return h

In [4]:
class ImprovedGRU(nn.Module):
    """
    改进版 GRU 模块，处理输入的整个时间序列
    输入 x 的形状：(batch, seq_len, input_size)
    输出所有时间步的隐藏状态，形状：(batch, seq_len, hidden_size)
    """
    def __init__(self, input_size, hidden_size, attn_size):
        super(ImprovedGRU, self).__init__()
        self.cell = ImprovedGRUCell(input_size, hidden_size, attn_size)

    def forward(self, x):
        batch_size, seq_len, _ = x.size()
        # 初始化隐藏状态，全零
        h = torch.zeros(batch_size, self.cell.hidden_size, device=x.device)
        outputs = []
        for t in range(seq_len):
            h = self.cell(x[:, t, :], h)
            outputs.append(h.unsqueeze(1))
        outputs = torch.cat(outputs, dim=1)  # (batch, seq_len, hidden_size)
        return outputs

In [5]:
#############################
# 2. Graph Attention Network (GAT) Module
#############################

class GraphAttentionLayer(nn.Module):
    """
    Graph Attention Layer
    通过自注意力 (Self-Attention) 机制动态计算节点之间的权重，
    并聚合邻居节点特征。参考公式：
      e_ij = LeakyReLU(a^T [Wh_i || Wh_j])
    """
    def __init__(self, in_features, out_features, dropout=0.1, alpha=0.2, concat=True):
        super(GraphAttentionLayer, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.dropout = dropout
        self.alpha = alpha
        self.concat = concat

        self.W = nn.Linear(in_features, out_features, bias=False)
        self.a = nn.Linear(2*out_features, 1, bias=False)
        self.leakyrelu = nn.LeakyReLU(self.alpha)

    def forward(self, h, adj):
        """
        h: 输入节点特征，形状 (N, in_features)，N为节点数（股票数）
        adj: 邻接矩阵，形状 (N, N)，非零表示存在边连接
        """
        Wh = self.W(h)  # (N, out_features)
        N = Wh.size()[0]
        # 构造所有节点对的拼接表示：[Wh_i || Wh_j]
        a_input = torch.cat([Wh.repeat(1, N).view(N*N, -1), Wh.repeat(N, 1)], dim=1)
        a_input = a_input.view(N, N, 2*self.out_features)
        # 计算注意力分数 e_ij
        e = self.leakyrelu(self.a(a_input).squeeze(2))  # (N, N)

        # 对非邻居设置为一个极小值
        zero_vec = -9e15 * torch.ones_like(e)
        attention = torch.where(adj > 0, e, zero_vec)
        attention = F.softmax(attention, dim=1)
        attention = F.dropout(attention, self.dropout, training=self.training)
        # 聚合邻居特征
        h_prime = torch.matmul(attention, Wh)  # (N, out_features)
        if self.concat:
            return F.elu(h_prime)
        else:
            return h_prime


In [6]:
#############################
# 3. Multi-head Cross-Attention Module
#############################

class MultiHeadCrossAttention(nn.Module):
    """
    Multi-head Cross-Attention 模块
    将 query (查询) 与 key, value (键、值) 进行交互，捕获不同子空间的信息。
    参考 Transformer 中的 Multi-head Attention 机制。
    """
    def __init__(self, d_model, num_heads):
        super(MultiHeadCrossAttention, self).__init__()
        assert d_model % num_heads == 0, "d_model 必须能被 num_heads 整除"
        self.num_heads = num_heads
        self.d_head = d_model // num_heads

        self.W_Q = nn.Linear(d_model, d_model)
        self.W_K = nn.Linear(d_model, d_model)
        self.W_V = nn.Linear(d_model, d_model)
        self.out_linear = nn.Linear(d_model, d_model)

    def forward(self, query, key, value):
        """
        query: (batch, seq_len_q, d_model)
        key, value: (batch, seq_len_k, d_model)
        输出: (batch, seq_len_q, d_model)
        """
        batch_size = query.size(0)
        Q = self.W_Q(query)  # (batch, seq_len_q, d_model)
        K = self.W_K(key)    # (batch, seq_len_k, d_model)
        V = self.W_V(value)  # (batch, seq_len_k, d_model)

        # 分头处理
        Q = Q.view(batch_size, -1, self.num_heads, self.d_head).transpose(1,2)  # (batch, num_heads, seq_len_q, d_head)
        K = K.view(batch_size, -1, self.num_heads, self.d_head).transpose(1,2)  # (batch, num_heads, seq_len_k, d_head)
        V = V.view(batch_size, -1, self.num_heads, self.d_head).transpose(1,2)  # (batch, num_heads, seq_len_k, d_head)

        # 计算缩放点积注意力
        scores = torch.matmul(Q, K.transpose(-2,-1)) / (self.d_head ** 0.5)  # (batch, num_heads, seq_len_q, seq_len_k)
        attn = F.softmax(scores, dim=-1)
        context = torch.matmul(attn, V)  # (batch, num_heads, seq_len_q, d_head)

        # 将多头拼接
        context = context.transpose(1,2).contiguous().view(batch_size, -1, self.num_heads * self.d_head)
        output = self.out_linear(context)  # (batch, seq_len_q, d_model)
        return output

In [7]:
#############################
# 4. 整体 MCI-GRU 模型
#############################

class MCI_GRU(nn.Module):
    """
    整体 MCI-GRU 模型，由以下四部分组成：
      I. Improved GRU 捕获时序特征 -> 输出 A1 (shape: (batch, hidden_size))
      II. GAT 捕获横截面特征 -> 输出 A2 (shape: (batch, gat_hidden))
      III. 多头交叉注意力捕获市场隐状态 -> 分别对 A1 与 A2 进行交互，输出 B1 和 B2 (shape: (batch, d_model))
      IV. 将 A1, A2, B1, B2 拼接后经过额外 GAT 层与全连接层得到最终预测
    """
    def __init__(self, input_size, hidden_size, attn_size,
                 gat_in, gat_hidden, d_model, num_heads, num_latent_states, final_out_dim):
        super(MCI_GRU, self).__init__()
        # 模块 I：Improved GRU
        self.improved_gru = ImprovedGRU(input_size, hidden_size, attn_size)
        # 模块 II：GAT
        self.gat = GraphAttentionLayer(gat_in, gat_hidden, dropout=0.1, alpha=0.2, concat=True)
        # 初始化市场隐状态向量（Learnable latent states）
        # R1 用于时序信息，R2 用于横截面信息；尺寸均为 (num_latent_states, d_model)
        self.R1 = nn.Parameter(torch.randn(num_latent_states, d_model))
        self.R2 = nn.Parameter(torch.randn(num_latent_states, d_model))
        # 模块 III：多头交叉注意力
        self.cross_attn1 = MultiHeadCrossAttention(d_model, num_heads)
        self.cross_attn2 = MultiHeadCrossAttention(d_model, num_heads)
        # 模块 IV：预测层——额外的 GAT 层和全连接层
        self.pred_gat1 = GraphAttentionLayer(final_out_dim, final_out_dim, dropout=0.1, alpha=0.2, concat=True)
        self.pred_gat2 = GraphAttentionLayer(final_out_dim, final_out_dim, dropout=0.1, alpha=0.2, concat=False)
        self.fc = nn.Linear(final_out_dim, 1)  # 最终预测输出一个标量（例如，未来收益率）

    def forward(self, x_time, x_graph, adj):
        """
        x_time: 时序数据，形状 (batch, seq_len, input_size)，每个样本对应一只股票的历史数据
        x_graph: 横截面数据，形状 (N, gat_in)，N 与 batch 大小相同，代表当前时刻每只股票的特征
        adj: 邻接矩阵，形状 (N, N)，描述股票之间的关系（例如，根据历史相关性构造）
        """
        # 模块 I：Improved GRU 提取时序特征，取最后一个时间步作为 A1，形状 (batch, hidden_size)
        gru_out = self.improved_gru(x_time)
        A1 = gru_out[:, -1, :]

        # 模块 II：GAT 提取横截面特征，输出 A2，形状 (N, gat_hidden)
        A2 = self.gat(x_graph, adj)

        # 模块 III：多头交叉注意力
        # 先对 A1 与 A2 添加时间步维度（视为序列长度为1）
        A1_unsq = A1.unsqueeze(1)  # (batch, 1, hidden_size)
        A2_unsq = A2.unsqueeze(1)  # (batch, 1, gat_hidden)
        # 为简化，要求 hidden_size, gat_hidden 均等于 d_model，此处假定已预设好
        # 与隐状态 R1 和 R2 进行交互
        R1_batch = self.R1.unsqueeze(0).expand(A1.size(0), -1, -1)  # (batch, num_latent_states, d_model)
        B1 = self.cross_attn1(A1_unsq, R1_batch, R1_batch)  # (batch, 1, d_model)
        B1 = B1.squeeze(1)  # (batch, d_model)

        R2_batch = self.R2.unsqueeze(0).expand(A2.size(0), -1, -1)  # (batch, num_latent_states, d_model)
        B2 = self.cross_attn2(A2_unsq, R2_batch, R2_batch)  # (batch, 1, d_model)
        B2 = B2.squeeze(1)  # (batch, d_model)

        # 拼接所有特征：A1, A2, B1, B2
        Z = torch.cat([A1, A2, B1, B2], dim=1)  # (batch, hidden_size + gat_hidden + 2*d_model)
        # 设 final_out_dim = hidden_size + gat_hidden + 2*d_model

        # 模块 IV：通过额外 GAT 层进一步整合特征并降维
        Z1 = self.pred_gat1(Z, adj)  # (batch, final_out_dim)
        Z2 = self.pred_gat2(Z1, adj)  # (batch, final_out_dim)
        out = self.fc(Z2)           # (batch, 1)
        return out

In [8]:
if __name__ == '__main__':
    # 设置随机种子，方便复现
    torch.manual_seed(0)

    # 参数设置（可根据需要调整）
    batch_size = 10       # 假设有10只股票
    seq_len = 10          # 历史时间步长度
    input_size = 6        # 每天6个财务指标
    hidden_size = 32      # Improved GRU 的隐藏层维度
    attn_size = 16        # 注意力机制内部维度
    gat_in = 6            # GAT 输入特征维度（与 input_size 一致）
    gat_hidden = 32       # GAT 输出维度
    d_model = 32          # 多头交叉注意力中使用的模型维度，假设与 hidden_size, gat_hidden 相同
    num_heads = 4         # 多头注意力的头数
    num_latent_states = 4 # 隐状态数量
    # 最终拼接后特征维度：hidden_size + gat_hidden + 2*d_model
    final_out_dim = hidden_size + gat_hidden + 2*d_model  # 32 + 32 + 64 = 128

    # 构造随机输入数据
    # 时序数据：形状 (batch, seq_len, input_size)
    x_time = torch.randn(batch_size, seq_len, input_size)
    # 横截面数据：形状 (batch, gat_in)；假设与 batch 大小相同
    x_graph = torch.randn(batch_size, gat_in)
    # 邻接矩阵：形状 (batch, batch)，简单构造一个全1的邻接矩阵（实际中应基于相关性过滤）
    adj = torch.ones(batch_size, batch_size)

    # 测试 Improved GRU 模块
    print("==== 测试 Improved GRU ====")
    gru = ImprovedGRU(input_size, hidden_size, attn_size)
    gru_out = gru(x_time)  # 输出形状: (batch, seq_len, hidden_size)
    print("ImprovedGRU 输出形状：", gru_out.shape)

    # 测试 GAT 模块
    print("\n==== 测试 GAT 模块 ====")
    gat_layer = GraphAttentionLayer(gat_in, gat_hidden, dropout=0.1, alpha=0.2, concat=True)
    gat_out = gat_layer(x_graph, adj)  # 输出形状: (batch, gat_hidden)
    print("GAT 输出形状：", gat_out.shape)

    # 测试 Multi-head Cross-Attention 模块
    print("\n==== 测试 Multi-head Cross-Attention 模块 ====")
    cross_attn = MultiHeadCrossAttention(d_model, num_heads)
    # 构造简单的 query 和 key/value 序列，均设形状 (batch, 1, d_model)
    query = torch.randn(batch_size, 1, d_model)
    key = torch.randn(batch_size, 1, d_model)
    value = torch.randn(batch_size, 1, d_model)
    attn_out = cross_attn(query, key, value)
    print("Multi-head Cross-Attention 输出形状：", attn_out.shape)

    # 测试整体 MCI_GRU 模型
    print("\n==== 测试整体 MCI_GRU 模型 ====")
    model = MCI_GRU(input_size, hidden_size, attn_size,
                    gat_in, gat_hidden, d_model, num_heads, num_latent_states, final_out_dim)
    pred = model(x_time, x_graph, adj)  # 输出形状: (batch, 1)
    print("整体模型预测输出形状：", pred.shape)

==== 测试 Improved GRU ====
ImprovedGRU 输出形状： torch.Size([10, 10, 32])

==== 测试 GAT 模块 ====
GAT 输出形状： torch.Size([10, 32])

==== 测试 Multi-head Cross-Attention 模块 ====
Multi-head Cross-Attention 输出形状： torch.Size([10, 1, 32])

==== 测试整体 MCI_GRU 模型 ====
整体模型预测输出形状： torch.Size([10, 1])


In [9]:
!pip install yfinance --quiet

import yfinance as yf
import pandas as pd
import datetime

In [15]:
def get_sp500_tickers():
    """
    从 Wikipedia 获取 S&P 500 成分股列表，并调整部分 ticker 格式（例如 BRK.B -> BRK-B）
    """
    url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
    tables = pd.read_html(url)
    df = tables[0]
    tickers = df['Symbol'].tolist()
    tickers = [ticker.replace('.', '-') for ticker in tickers]
    return tickers

In [11]:
#######################################
# 2. 获取单个股票数据
#######################################

def fetch_stock_data(ticker, start_date, end_date):
    """
    利用 yfinance 获取单个股票的历史数据
    参数：
      ticker: 股票代码（例如 'AAPL'）
      start_date, end_date: 数据时间范围，格式 'YYYY-MM-DD'
    返回：
      数据 DataFrame，包含 Open, High, Low, Close, Volume 等字段
    """
    stock = yf.Ticker(ticker)
    df = stock.history(start=start_date, end=end_date)
    # 注意：Yahoo Finance 默认不提供 turnover 数据，这里仅返回已有字段
    return df

In [20]:
def count_trading_days(start_date, end_date):
    """
    利用 pandas 计算在 start_date 到 end_date 内的预期交易日数量（仅考虑工作日，freq='B'）
    """
    rng = pd.date_range(start=start_date, end=end_date, freq='B')
    return len(rng)

In [27]:
import os

In [29]:
#######################################
# 2. 数据完整性检查函数
#######################################

def has_full_data(ticker, train_start, train_end, val_start, val_end, test_start, test_end, tolerance=0.8):
    """
    检查单个股票 ticker 在训练、验证和测试三个时间段内是否具备足够数据覆盖。
    方法：计算各时段预期的交易日数（工作日数），要求实际数据行数不少于预期的 tolerance 倍。
    """
    try:
        expected_train = count_trading_days(train_start, train_end)
        expected_val = count_trading_days(val_start, val_end)
        expected_test = count_trading_days(test_start, test_end)

        df_train = fetch_stock_data(ticker, train_start, train_end)
        df_val = fetch_stock_data(ticker, val_start, val_end)
        df_test = fetch_stock_data(ticker, test_start, test_end)
    except Exception as e:
        print(f"获取 {ticker} 数据时出错: {e}")
        return False

    # 若任一时段数据为空，则直接返回 False（利用 Yahoo API 返回结果）
    if df_train.empty or df_val.empty or df_test.empty:
        return False

    if df_train.shape[0] < tolerance * expected_train:
        return False
    if df_val.shape[0] < tolerance * expected_val:
        return False
    if df_test.shape[0] < tolerance * expected_test:
        return False

    return True

def get_common_tickers(tickers, train_start, train_end, val_start, val_end, test_start, test_end, tolerance=0.8):
    """
    对给定的 tickers 列表，返回在训练、验证和测试三个时间段内数据完整（满足 tolerance 要求）的股票列表
    """
    common = []
    for ticker in tickers:
        if has_full_data(ticker, train_start, train_end, val_start, val_end, test_start, test_end, tolerance):
            common.append(ticker)
        else:
            print(f"股票 {ticker} 数据不完整，已剔除。")
    return common

In [30]:
#######################################
# 3. 保存有效股票列表及数据的函数
#######################################

def save_valid_tickers_and_data(output_dir, train_start, train_end, val_start, val_end, test_start, test_end, tolerance=0.8):
    """
    1. 根据指定时间区间筛选出数据完整的股票列表（即在训练、验证、测试集均具备足够数据的股票）。
    2. 将有效股票列表保存到 output_dir（例如保存为 valid_tickers.csv）。
    3. 对于每个有效股票，从 2018-01-01 至 2023-12-31 的全数据进行查询，并保存为 CSV 文件，方便以后调用。

    参数：
      output_dir: 输出目录，若不存在则自动创建
      train_start, train_end, val_start, val_end, test_start, test_end: 时间区间，格式 'YYYY-MM-DD'
      tolerance: 容忍度，例如 0.8 表示实际交易日数不少于预期的 80%
    """
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    all_tickers = get_sp500_tickers()
    print(f"候选股票总数：{len(all_tickers)}")

    # 筛选出在训练、验证和测试集都有足够数据的股票
    valid_tickers = get_common_tickers(all_tickers, train_start, train_end, val_start, val_end, test_start, test_end, tolerance)
    print(f"数据完整的股票数量：{len(valid_tickers)}")

    # 保存有效股票列表到 CSV 文件
    valid_tickers_df = pd.DataFrame(valid_tickers, columns=["Ticker"])
    tickers_file = os.path.join(output_dir, "valid_tickers.csv")
    valid_tickers_df.to_csv(tickers_file, index=False)
    print(f"有效股票列表已保存到: {tickers_file}")

    # 对于每个有效股票，获取从 2018-01-01 至 2023-12-31 的全数据，并保存为 CSV 文件
    full_start, full_end = '2019-01-01', '2024-12-31'
    for ticker in valid_tickers:
        print(f"保存 {ticker} 的全数据...")
        df = fetch_stock_data(ticker, full_start, full_end)
        if df.empty:
            print(f"股票 {ticker} 在全数据时间区间内数据为空，跳过保存。")
            continue
        file_path = os.path.join(output_dir, f"{ticker}.csv")
        df.to_csv(file_path)
    print("所有有效股票数据保存完成。")
    return valid_tickers

In [31]:
#######################################
# 4. 调试函数：检查有效股票列表及保存情况
#######################################

def debug_save_valid_data():
    """
    调试函数，调用 save_valid_tickers_and_data 检查有效股票列表并保存数据
    """
    # 定义论文中规定的时间区间
    train_start, train_end = '2019-01-01', '2022-12-31'
    val_start, val_end = '2023-01-01', '2023-12-31'
    test_start, test_end = '2024-01-01', '2024-12-31'

    # 输出目录（可自定义，例如 "./data/sp500"）
    output_dir = "./data/sp500"

    valid_tickers = save_valid_tickers_and_data(output_dir, train_start, train_end, val_start, val_end, test_start, test_end, tolerance=0.7)
    print("最终有效股票列表：", valid_tickers)

In [32]:
if __name__ == '__main__':
    print("开始检查训练、验证和测试数据中是否具有相同的节点（股票），并保存有效股票列表及数据...")
    debug_save_valid_data()
    print("检查与保存完成。")

开始检查训练、验证和测试数据中是否具有相同的节点（股票），并保存有效股票列表及数据...
候选股票总数：503
股票 ABNB 数据不完整，已剔除。
股票 CARR 数据不完整，已剔除。
股票 CEG 数据不完整，已剔除。
股票 GEHC 数据不完整，已剔除。


ERROR:yfinance:$GEV: possibly delisted; no price data found  (1d 2019-01-01 -> 2022-12-31) (Yahoo error = "Data doesn't exist for startDate = 1546318800, endDate = 1672462800")
ERROR:yfinance:$GEV: possibly delisted; no price data found  (1d 2023-01-01 -> 2023-12-31) (Yahoo error = "Data doesn't exist for startDate = 1672549200, endDate = 1703998800")


股票 GEV 数据不完整，已剔除。


ERROR:yfinance:$KVUE: possibly delisted; no price data found  (1d 2019-01-01 -> 2022-12-31)


股票 KVUE 数据不完整，已剔除。


...

股票 SW 数据不完整，已剔除。


ERROR:yfinance:$SOLV: possibly delisted; no price data found  (1d 2019-01-01 -> 2022-12-31) (Yahoo error = "Data doesn't exist for startDate = 1546318800, endDate = 1672462800")
ERROR:yfinance:$SOLV: possibly delisted; no price data found  (1d 2023-01-01 -> 2023-12-31) (Yahoo error = "Data doesn't exist for startDate = 1672549200, endDate = 1703998800")


股票 SOLV 数据不完整，已剔除。


ERROR:yfinance:$VLTO: possibly delisted; no price data found  (1d 2019-01-01 -> 2022-12-31) (Yahoo error = "Data doesn't exist for startDate = 1546318800, endDate = 1672462800")


股票 VLTO 数据不完整，已剔除。
数据完整的股票数量：491
有效股票列表已保存到: ./data/sp500/valid_tickers.csv
保存 MMM 的全数据...
保存 AOS 的全数据...
保存 ABT 的全数据...
保存 ABBV 的全数据...
保存 ACN 的全数据...
保存 ADBE 的全数据...
保存 AMD 的全数据...
保存 AES 的全数据...
保存 AFL 的全数据...
保存 A 的全数据...
保存 APD 的全数据...
保存 AKAM 的全数据...
保存 ALB 的全数据...
保存 ARE 的全数据...
保存 ALGN 的全数据...
保存 ALLE 的全数据...
保存 LNT 的全数据...
保存 ALL 的全数据...
保存 GOOGL 的全数据...
保存 GOOG 的全数据...
保存 MO 的全数据...
保存 AMZN 的全数据...
保存 AMCR 的全数据...
保存 AEE 的全数据...
保存 AEP 的全数据...
保存 AXP 的全数据...
保存 AIG 的全数据...
保存 AMT 的全数据...
保存 AWK 的全数据...
保存 AMP 的全数据...
保存 AME 的全数据...
保存 AMGN 的全数据...
保存 APH 的全数据...
保存 ADI 的全数据...
保存 ANSS 的全数据...
保存 AON 的全数据...
保存 APA 的全数据...
保存 APO 的全数据...
保存 AAPL 的全数据...
保存 AMAT 的全数据...
保存 APTV 的全数据...
保存 ACGL 的全数据...
保存 ADM 的全数据...
保存 ANET 的全数据...
保存 AJG 的全数据...
保存 AIZ 的全数据...
保存 T 的全数据...
保存 ATO 的全数据...
保存 ADSK 的全数据...
保存 ADP 的全数据...
保存 AZO 的全数据...
保存 AVB 的全数据...
保存 AVY 的全数据...
保存 AXON 的全数据...
保存 BKR 的全数据...
保存 BALL 的全数据...
保存 BAC 的全数据...
保存 BAX 的全数据...
保存 BDX 的全数据...
保存 BRK-B 的全数据...
保存 BBY 